## Define Baseline Period

In [ ]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, show, output_file, save, curdoc
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter, Title
from bokeh.models.widgets import Select
from bokeh.layouts import column, row, gridplot
from datetime import datetime as dt
from math import pi, sqrt
import os
import csv
import statistics

### Loads the files

In [ ]:
# 1. Load the dato into dataframe
def read_file(file_name):
    my_absolute_dirpath = os.path.abspath(os.path.dirname(file_name))
    file_path = my_absolute_dirpath+"\\aResults\\Baseline\\"+file_name
    df = pd.read_csv(file_path, sep=",")
    return df

In [ ]:
# 1. Loads all the files
def load_all_files():
    
    baseline_dfs = []
    
    for participant in ['01', '02', '04', '05', '06']:
        for i in range(1,4):
            file_name = 'baseline_'+participant+'_'+str(i)+'.csv'
            baseline_dfs.append(read_file(file_name))
            
    return baseline_dfs

### Calculates the best baseline

In [ ]:
# Method which defines the best window
def relaxed_window(baseline_dfs):
    
    # Defining variable for the windows comparison
    min_sum = 1000
    min_second = 60
    
    # Iterating through the windows
    for second in range (0, 50):
        
        # Defining sum of all ratios in given window
        mean_sum = 0
     
        # Iterating through the baselines
        for df_baseline in baseline_dfs:
            
            # Casting timestemp as a datetime
            df_baseline['TimeStamp'] = pd.to_datetime(df_baseline['TimeStamp'], errors='coerce')

            # Defining start_timestamp and end_timestamp
            start_timestamp = df_baseline.iloc[0]['TimeStamp'] + pd.Timedelta(seconds=second)
            end_timestamp = start_timestamp + pd.Timedelta(seconds=10)

            # Defining the window
            df_ten_seconds_window = df_baseline[df_baseline['TimeStamp'] >= start_timestamp]
            df_ten_seconds_window = df_ten_seconds_window[df_ten_seconds_window['TimeStamp'] < end_timestamp]

            # Calculating the baseline
            first_ratio_mean = df_ten_seconds_window['First_Ratio'].mean()
            second_ratio_mean = df_ten_seconds_window['Second_Ratio'].mean()

            mean_sum = mean_sum + first_ratio_mean + second_ratio_mean

        # Comparing minimum value
        if mean_sum < min_sum:
            min_sum = mean_sum
            min_second = second
            
        #print("Second: " +str(second)+", sum of ratios: "+str(mean_sum)+". Best value at second: "+str(min_second)+", sum: "+str(min_sum))

    return min_second, min_sum

In [ ]:
# Execution
baseline_dfs = load_all_files()
second, sum = relaxed_window(baseline_dfs)
print("Best result: Second "+str(second)+" with value "+str(sum))